## Gradient methods

#### Proplem:
$$
f(\vec{x}) \rightarrow min,\\
f: \Omega \rightarrow \mathbb{R}, \\
\Omega \subset \mathbb{R^n}, f(\vec{x}) \mbox{ is convex}, \\
f(\vec{x}) \mbox{ - is diffirentiable on } \Omega\\
\vec{x_*} \in \Omega, f_{min} = f(\vec{x_*})
$$

<em>**Definition**</em>.

Sequnce $\{\vec{x_k}\}$ is named **Relaxational**, if $\forall k \in \mathbb{N}:  f(\vec{x_k}) < f(\vec{x}_{k-1})$ 

$\{\vec{x}_l\}$ convergece to $\vec{x}_* \in \mathbb{R}^n$ by Bolzano–Weierstrass theorem 

Let's choose our relaxational sequence by this equation:
$$
\vec{x}_k = \vec{x}_{k-1} + \beta_k\vec{u}_k
$$
where $\vec{u}_{k}$ is unit vector, which defines the direction of descent and $\beta_k \geq 0$ - length of descent step

<em>**Lemma**</em>.

$f(\vec{x})$ - is differentiable on $\Omega \subset \mathbb{R}^n$ and $\exists L > 0$, such that $\forall \vec{x}, \vec{y} \in \Omega$:
$$
||\nabla f(\vec{x}) - \nabla f(\vec{y})|| \leq  L ||\vec{x} = \vec{y}|| 
$$
Then:
$$
f(\vec{x}) - f(\vec{y}) \geq (\nabla f(\vec{x}), \vec{x} - \vec{y}) - \frac{L}{2}||\vec{x}-\vec{y}||^2
$$
<em>**Definition**</em>.

$\vec{w}(\vec{x}) = - \nabla f(\vec{x})$ is called **antigradient**

If we take our $\vec{u}_k = \frac{\vec{w}_k}{||\vec{w}_k||}$, from our lemma we have, that: 

$$
f(x_{k}) - f(x_{k+1}) \geq (\nabla f(x_k), \vec{x_k} - \vec{x_k} - \beta_k \frac{\vec{w_k}}{||\vec{w_k}||}) - \frac{L}{2} || \vec{x_k} - \vec{x_k} - \beta_k \frac{\vec{w_k}}{||\vec{w_k}||} ||^2 = \beta_k||\nabla f(\vec{x}_k)|| - \beta_k \frac{L}{2} 
$$
As we can see gradient must be always posistive (and $> \frac{L}{2}$),  so that we have a convergece, we get this when function is convex

All methods in which $\vec{u}_k = \frac{\vec{w}_k}{||\vec{w}_k||}$, are named ***gradient methods***, the methods vary on the way we choose our $\beta_k > 0$




In [3]:
import matplotlib as mplib
import math as m
import numpy as np
from numpy.linalg import norm

from onedim_optimize import newton_method
from scipy.optimize import approx_fprime

def toOneParamFunc(f, x, w):
    return lambda p: f(x + p*w) 

def argmin(f, x, eps):
    f_x, xmin, k = newton_method(f, x, eps)
    return xmin, k

def approx_gradient(f, eps):
    return lambda x: approx_fprime(x, f, eps)

### Test functions

#### Rosenbrock banana function:
$$
f(x_1, x_2, ..., x_N) = \sum^{N/2}_{i=1}[100(x^2_{2i-1} - x_{2i})^2 + (x_{2i-1} - 1)^2]
$$

In [ ]:
def rosenbrock(x):
    build_ros_terms = lambda vec: x.map
    return lambda x: reduce(func, x)

### Fastest descent method

We will construct relaxational sequence, using this rule:
$$
\vec{x}_{k+1} = \vec{x}_k + \lambda_k\vec{w}_K
$$

Where $\lambda_k$ is found from
$$
\lambda_k = argmin\{\psi_k(\lambda)\} \\
\psi_k(\lambda) = f(\vec{x}_{k-1} + \lambda\vec{w}_k)
$$

Finding minimum of $\psi_k(\lambda)$ is a pretty complex task of one-dimension minimization. But it is guaranteed that $\{|\vec{w}_k|\}$ convergace to 0.

So at start we pick some small $\epsilon$ and continuing procedure while $|\vec{w}_k\| > \epsilon$, than on some N iteration we pick our $x_* = x_N$

In [4]:
def fastest_descent(f, gr, x, epsilon):
    n = len(x)
    w = -gr(x) 
    phi = toOneParamFunc(f, x, w)
    l, i = argmin(phi, 0, epsilon)
    n += i
    k = 1
    print(x, f(x), l, norm(w))
    x = x + l*w
    while(norm(w) > epsilon):
        w = -gr(x) 
        phi = toOneParamFunc(f, x, w)
        l, i = argmin(phi, 0, epsilon)
        n += i
        k += 1
        print(x, f(x), l, norm(w))
        x = x + l*w
    return f(x), x, k, n

In [20]:
f1 = lambda x: 6*x[0]**2 - 4*x[0]*x[1] + 3*x[1]**2 + 4*m.sqrt(5)*(x[0] + 2*x[1]) + 22
f2 = lambda x: (x[0]**2 - x[1])**2 + (x[0] - 1)**2

test1 = [
    f1,
    approx_gradient(f1, 1e-8),
    np.array([-2, 1]),
    0.01,
]
test2 = [
    f2,
    approx_gradient(f2, 1e-8),
    np.array([-1, -2]),
    0.001,
]

gradient = approx_gradient(test, 1e-8)
fmin, xmin, K, N = fastest_descent(*test1)
print(f"""
x minimum: {xmin},
f minimum: {fmin},
number of iterations: {K}
number of one-dimension minimization iterations: {N}
""")

[-1 -2] 13 0.08615979586424602 17.08800732430959
[ 0.37855672 -1.48304122] 3.031194410271926 0.33477672577953216 3.473881365831169
[-0.02979422 -0.39411538] 1.2165035678374376 0.2941390100226454 2.2499202860491687
[ 0.58985773 -0.16174376] 0.42798620486760236 0.2584189708764911 1.0886694780047377
[0.49107435 0.10167608] 0.278459413443343 0.27187016106120226 0.7944599505351536
[0.69331181 0.17751586] 0.1859669109978042 0.23312279277088116 0.6475623544330313
[0.64030522 0.31886538] 0.1376841732552048 0.24263049498239633 0.5190459669552201
[0.75822298 0.36308498] 0.10332261616893182 0.21937038274234105 0.4524420985482393
[0.72337283 0.45601778] 0.08104521598502114 0.22334395023759432 0.3830555194902436
[0.80347863 0.48605775] 0.06406732979570523 0.21055433552507619 0.34073566558359303
[0.7782876  0.55323307] 0.05191248179446088 0.2099527933707892 0.29902875145012625
[0.83707207 0.57527749] 0.042273719770566404 0.20437179805116157 0.2678808582982234
[0.81784881 0.6265389 ] 0.03497154282313

### Conjugate gradient method

#### Problem 

$$
f(\vec{x}) = \frac{1}{2}(Q\vec{x}, \vec{x}) + (\vec{c}, \vec{x}) \rightarrow min
$$

$Q$ is positive determined n-dimsensional matrix, $c \in \mathbb{R}$ - constant

This function has single point of minimum $x_* = -Q^{-1}\vec{c}$

To find the inverted matrix $Q^{-1}$ we can use
$$
Q^{-1} = \sum^n_{i=1}\frac{p^i(p^i)^T}{(Qp^i, p^i)}
$$
Where $p^i \in \mathbb{R}$ is conjugate vector of matrix $Q$

But constructing a system of conjugate vectors is a pretty complex problem.

So we do another way, let's construct system of conjugate vectors on every iteration

$\vec{x}_0$ is a starting point, antrigradient in this point is $\vec{w}_1 = -Qx_0 - c$ and let's choose $\vec{p}_1 = \vec{w}$

Using $\vec{x}_k = \vec{x}_{k-1} + \lambda_k\vec{w}_k$

We can find that 
$$\lambda_1 = \frac{|\vec{w}_1|^2}{(Q\vec{w}_1, \vec{w}_1)} = \frac{|\vec{p}_1|^2}{(Q\vec{p}_1, \vec{p}_1)}$$
(from minimization of quadratic function)

And so $x_1 = x_0 + \lambda_1\vec{p}_1$

On second iteration (k = 2) we evaluate antigradient $\vec{w}_2 = -Q\vec{x_1} - c$

Let's assume, that
$$\vec{p}_2 = \gamma_1\vec{p}_1 + \vec{w}_2$$

If we product scalarly this equation on $Q\vec{p}_1 \not = 0$ and demand that $\vec{p}_1, \vec{p}_2$ are conjugate (ortogonal) over the matrix $Q$ ($(Q\vec{p}_1, \vec{p_2}) = 0$), we can find $\gamma_1$
$$\gamma_1 = -\frac{(Q\vec{p}_1, \vec{w}_2)}{(Q\vec{p}_1, \vec{p}_1)}$$

Contniuing constructing this system of conjugate vectors, we can say, that on every k iteration we have system of equations:
$$
\begin{cases}
    p_{k+1} = \gamma\vec{p_k} + \vec{w}_{k+1} \\
    \gamma_k = - \frac{(Q\vec{p}_k, \vec{w}_{k+1})}{(Q\vec{p}_k, \vec{p}_k)} \\
    \vec{w}_{k+1} = \vec{w}_k = \lambda_kQ\vec{p}_k \\
    (Q\vec{p}_{k+1}, \vec{p}_i) = 0 \\
    (\vec{w}_{k+1}, \vec{w}_i) = 0, i = \overline{1, k} \\
\end{cases} \\
\mbox{also } \\
\lambda_k = \frac{(\vec{w}_k, \vec{p}_k)}{(Q\vec{p}_k, \vec{p}_k)},\\
\vec{x}_k = \vec{x_1} + \lambda_k\vec{p}_k
$$

With n steps we can find all $\vec{p}_k$ conjugate vectors and evaluate our minimum $x_* = -Q^{-1}\vec{c}$

To use this method in our problems (non-quadratic function optimization, we need to remove matrix $Q$ from system of equations

We can do this, by if on every iteration by doing minimization process:
$$
\psi_k(\lambda) = f(x_{k-1} + \lambda)
$$

In fundament of constructing conjuguate directions $\vec{p}_{k+1} = \gamma_k\vec{p}_k + \vec{w}_{k+1}$ we assume, that $(\vec{w}_{k+1}, \vec{w}_i) = 0$

Using this we can show that:
$$
\begin{cases}
    (Q\vec{p}_k, \vec{w}_{k+1}) = - \frac{1}{\lambda_k}|\vec{w}_{k+1}|^2 \\
    (Q\vec{p}_k, \vec{p}_{k}) = \frac{1}{\lambda_k}(\vec{w}_k, \vec{p}_k)
\end{cases} \\
\mbox{so from our system of equations we can evaluate $\gamma$ using one of theese formulas: } \\
\gamma_k = \frac{|\vec{w}_{k+1}|^2}{|\vec{w}_k|^2} \\
\gamma_k = \frac{(\vec{w}_{k+1} - \vec{w}_k, \vec{w}_{k+1})}{|\vec{w}_k|^2} \\
\mbox{also if function twice differentiable, we can use Hessian instead of matrix Q:} \\
\gamma_k = - \frac{(H(\vec{x}_k)\vec{p}_k, \vec{w}_{k+1})}{(H(\vec{x}_k)\vec{p}_k, \vec{p}_k)} \\
$$

This method is called ***conjaguate gradients method***

Also as every $\gamma_k$ is different and we need to minimize $\psi_k(\lambda)$ this turns us to inevitably errors, to minimize errors, we need to do **restarts** (set $\gamma_k = 0$). It is common to restart every $n$ times, where $n$ is our dimension number. Also, with non-quadratic functions our procedure of optimization in general don't take $n$ steps, so we choose our $\epsilon$ and iterate through $\{\vec{x}_k\}$ till our |$\vec{w}_{k+1|} < \epsilon$, and then $x_{k-1} \approx x_*$ 



In [25]:
def conjugate_gradient(f, gr, x, epsilon):
    w = -gr(x) 
    p = w
    phi = toOneParamFunc(f, x, p)
    l, i = argmin(phi, 0, epsilon)
    n = i
    print(x, f(x), l, norm(w))
    x = x + l*p
    k = 1
    while norm(p) > epsilon:
        w_2 = -gr(x)
        gamma = 0
        if k % n != 0:
            gamma = (np.dot(w_2 - w, w_2))/norm(w)**2
        p = gamma*p + w_2
        phi = toOneParamFunc(f, x, p)
        l, i = argmin(phi, 0, epsilon) 
        n += i
        print(x, f(x), l, norm(w_2))
        x = x + l*p
        w = w_2
        k += 1
    return f(x), x, k, n
        

In [26]:
f1 = lambda x: 6*x[0]**2 - 4*x[0]*x[1] + 3*x[1]**2 + 4*m.sqrt(5)*(x[0] + 2*x[1]) + 22
f2 = lambda x: (x[0]**2 - x[1])**2 + (x[0] - 1)**2

test1 = [
    f1,
    approx_gradient(f1, 1e-8),
    np.array([-2, 1]),
    0.01,
]
test2 = [
    f2,
    approx_gradient(f2, 1e-8),
    np.array([-1, -2]),
    0.001,
]

fmin, xmin, K, N = conjugate_gradient(*test2)
print(f"""
x minimum: {xmin},
f minimum: {fmin},
number of iterations: {K}
number of one-dimension minimization iterations: {N}
""")

[-1 -2] 13 0.08615979586424602 17.08800732430959
[ 0.37855672 -1.48304122] 3.031194410271926 0.39429340464599333 3.473881365831169
[ 0.15834029 -0.10275169] 0.7247298758120327 0.4906820879947748 1.6226263306501927
[0.85929719 0.55729545] 0.052593117788155466 0.2896826851184612 0.4974964403391633
[0.87074064 0.76602386] 0.016769364424484637 0.3915138203121558 0.2862355375850555
[0.9942033  0.97084451] 0.000343210345130694 0.1459897693910945 0.0681676619727319
[0.99797699 0.99638207] 4.272342498360597e-06 0.25380186009263545 0.0058008328114193455
[0.99999607 0.99997434] 3.323150132996604e-10 0.12582217179919694 7.26986869561118e-05

x minimum: [0.99999997 0.99999994],
f minimum: 1.0093783894492261e-15,
number of iterations: 8
number of one-dimension minimization iterations: 22

